In [1]:
from nltk.corpus import brown
from collections import Counter
import random

corpus = brown.tagged_words(tagset = 'universal') #creates tagged corpus

def find_rhyme_pos(string1):
    """
    Uses vowels to find the position of the rhyme in a word.
    
    Argument
    ---------
    string1: line of poetry
    
    Returns
    ---------
    length of the last word starting from the last vowel.
    
    Example
    ---------
    find_rhyme_pos("chicken")
    >>>2
    """

    vowels = ['a', 'e', 'i', 'o', 'u']
    for n in range(1, len(string1) - 1):
        if string1[-n] in vowels and string1[-(n+1)] not in vowels:
            return n
    return len(string1)

def rhymes(string1, string2): # checks rhyme
    """
    Checks if words at the end of the sentence that rhyme with the first sentence.
    
    Arguments
    ---------
    string1: 1st line of poetry
    string2: Other line of poetry
    
    Returns
    ---------
    Bool
    True if they rhyme, False otherwise.
    
    Example
    ---------
    rhymes("block", "shock")
    >>>True
    """
    return string1[-find_rhyme_pos(string1):] == string2[-find_rhyme_pos(string2):]

def construct_rhyme_dict(corpus): # makes whole dictionary of rhymes
    """
    Creates a dictionary of rhymes.
    
    Argument
    ---------
    corpus: corpus of poetry we are using.
    
    Returns
    ---------
    A dictionary with a word as a key and words that rhyme with that word as its values.
    
    Example
    ---------
    construct_rhyme_dict("Friday")
    >>>[('may', 'VERB'), ('Wednesday', 'NOUN'), ('Tuesday', 'NOUN'), ('Highway', 'NOUN'), ('Tuesday',     'NOUN'), ('Monday', 'NOUN'), ('highway', 'NOUN'), ('Monday', 'NOUN'), ('highway', 'NOUN'), ('Highway', 'NOUN'), ('highway', 'NOUN'), ('Highway', 'NOUN')]
    """
    rhyme_dict = {}
    for item1 in corpus:
        rhyme_dict[item1[0]] = []
        for item2 in corpus:
            if rhymes(item1[0], item2[0]) and item1 != item2:
                rhyme_dict[item1[0]].append(item2)
    return rhyme_dict

nouns = []
verbs = []
dets = []
adj = []
bigrams = []

#Dictionary that simplifies code
pos_dict = {'NOUN': nouns, 'ADJ': adj, 'VERB': verbs, 'DET': dets}

def pick_word(pos, syllables = 0):
    
    """
    Picks random word based on POS and syllables.
    
    Arguments
    ---------
    pos: part of speech needed in template.
    syllables: amount of syllables allowed in a particular area in the template.
    
    Returns
    ---------
    string
    
    Example
    ---------
    pick_word("ADJ")
    >>>"pretty"
    """

    for i in range(len(pos_dict[pos])):
        word = random.choice(pos_dict[pos])
        if syllables != 0 and syllable_count(word) == syllables:
            return word
    return random.choice(pos_dict[pos])

def bigram_probabilities(word, pos, syllables): # returns most probable word to come before, checks that its POS is correct
    """
    Returns most probable word to come before a word. Checks if POS is correct.
    
    Arguments
    ---------
    word: word in the line.
    pos: part of speech of that word.
    syllables: syllables allowed for that word.
    
    Returns
    ---------
    string
    
    Example
    ---------
    bigram_probabilities("block", NOUN, 1)
    >>>"big"
    """
    completions = []
    for pair in bigrams:
        if pair[1][0] == word: # if second word of bigram equals current word
            if pair[0][1] == pos: # if pos of first word in bigram equals part of speech needed
                if syllable_count(pair[0][0]) == syllables: # first word in bigram is number of syllables needed
                    completions.append(pair[0][0])
    if completions==[]: # if no words fit completion, have it return one with correct pos and syllable count:
        return pick_word(pos, syllables)
    completions = Counter(completions)
    return ((completions.most_common(1))[0][0])

def gather_words(): 
    """
    Makes groups of nouns, adjectives, verbs, etc.
    
    Returns
    ---------
    Lists with different words based on parts of speech
    
    Example
    ---------
    gather_words()
    >>>NOUNS = ["dog", "cat"]
       VERBS = ["run" , "jump"]
       ADJ   = ["nice", "mean"]
    """
    for word in range(1000):
        if pos_dict.get(corpus[word][1]) != None:
            pos_dict[corpus[word][1]].append(corpus[word][0])

def bigram_collector(corpus):
    """
    Creates list of word bigrams.
    
    Arguments
    ---------
    corpus: corpus we are using.
    
    Returns
    ---------
    list
    
    Example
    ---------
    bigram_collector(corpus)
    >>>[(('The', 'DET'), ('Fulton', 'NOUN')),
        (('Fulton', 'NOUN'), ('County', 'NOUN')),
        (('County', 'NOUN'), ('Grand', 'ADJ')),
        (('Grand', 'ADJ'), ('Jury', 'NOUN')),
    """
    for n in range(0, 4000):
        pair = (corpus[n], corpus[n+1])
        bigrams.append(pair)
    return bigrams

def syllable_count(word):
    """
    Counts number of syllables in word.
    
    Arguments
    ---------
    word: str
        a word in a line
        
    Returns
    ---------
    counter, number of syllables
    
    Example:
    ---------
    syllable_count("chicken")
    >>> 2
    
    """
    vowels = ['a', 'e', 'i', 'o', 'u', 'y', 'A', 'E', 'I', 'O', 'U', 'Y']
    count = 0
    boolean = True
    for char in range(0, len(word)):
        if word[char] not in vowels:
            boolean = True
        elif boolean and word[char] == 'e' and char == len(word) - 1:
            if (len(word) > 2 and word[char - 2] not in vowels) or len(word) <= 2:
                count += 1
        elif boolean and word[char] in vowels:
            count += 1
            boolean = False
    return count

template = [[('DET', 1), ('NOUN', 2), ('VERB', 1), ('ADJ', 1), ('NOUN', 1)],
                [('NOUN', 1), ('VERB', 1), ('DET', 1), ('ADJ', 2), ('NOUN', 1)],
                [('DET', 1), ('ADJ', 1), ('NOUN', 1), ('VERB', 1), ('ADJ', 1), ('NOUN', 1)],
                [('ADJ', 2), ('NOUN', 1), ('VERB', 1), ('ADJ', 1), ('NOUN', 1)]]


LookupError: ignored

In [0]:
def write_poem(template):
    """
    Writes the entire poem.
    
    Arguments
    ---------
    template: pre-defined templates that contain part of speech and syllable allowance.
    
    Returns
    ---------
    lists within a list
    
    Example
    ---------
    write_poem(template)
    >>>A Allen 

    birth have any greater manner 

    any mayor said mental Miller 

    Mayor told The other juries 

    any Jail been recent authorities
    """

    write_title()
    print('\n')
    line_1 = write_line(template[random.randint(0,3)], pick_word('NOUN', syllables = 2))
    rhyming_words = rhyme_dict[line_1[-1:][0]] # lists words that rhyme
    #if rhyming_words==[]:
        #line_2 = write_line(template[random.randint(0,3)], pick_word('NOUN'))
    #else:
        #line_2 = write_line(template[random.randint(0,3)], rhyming_words[0][0])
    if rhyming_words != []:
        for word in rhyming_words:
            if word[1] == 'NOUN':
                line_2 = write_line(template[random.randint(0,3)], word[0])
    else:
        line_2 = write_line(template[random.randint(0,3)], pick_word('NOUN', syllables = 2))
    poem = []
    poem.append(line_1)
    poem.append(line_2)
    line_3 = write_line(template[random.randint(0,3)], pick_word('NOUN', syllables = 2))
    rhyming_words = rhyme_dict[line_3[-1:][0]] # lists words that rhyme
    if rhyming_words != []:
        for word in rhyming_words:
            if word[1] == 'NOUN':
                line_4 = write_line(template[random.randint(0,3)], word[0])
    else:
        line_4 = write_line(template[random.randint(0,3)], pick_word('NOUN', syllables = 2))
    poem.append(line_3)
    poem.append(line_4)
    for line in poem:
        for word in line:
            print(word, end = " ")
        print('\n')

def write_line(sentence, last_word): # have to find a way to make this not hard coded
    """
    Writes the lines of the poem.
    
    Arguments
    ---------
    sentence: str
    last_word: the last word of the line, str
    
    Returns
    ---------
    list
    
    Example
    ---------
    write_poem(template)
    >>>
    """
    first_line=[]
    word = (bigram_probabilities(last_word, sentence[-2:][0][0], sentence[-2:][1][1])) # picks next four words
    third_word = (bigram_probabilities(word, sentence[-3:][0][0], sentence[-3:][1][1]))
    fourth_word = (bigram_probabilities(third_word, sentence[-4:][0][0], sentence[-4:][1][1]))
    fifth_word = (bigram_probabilities(fourth_word, sentence[-5:][0][0], sentence[-5:][1][1]))

    first_line.append(fifth_word) # adds four words to line in order
    first_line.append(fourth_word)
    first_line.append(third_word)
    first_line.append(word)
    first_line.append(last_word)
    return first_line

def write_title():
    """
    Writes a title for the poem.
    
    
    Returns
    ---------
    str
    
    Example
    ---------
    write_title()
    >>>A Allen
    """
    title_templates = [[('DET'), ('NOUN')], [('DET'), ('ADJ'), ('NOUN')], [('ADJ'), ('NOUN')], [('NOUN')], [('ADJ')], [('VERB')]]
    title = []
    for word in title_templates[random.randint(0,5)]: # picks words for each pos in title
        title.append(pick_word(word).capitalize())

    for word in title: #prints nicely
        print(word, end = " ")

template = [[('DET', 1), ('NOUN', 2), ('VERB', 1), ('ADJ', 1), ('NOUN', 2)],
                [('NOUN', 1), ('VERB', 1), ('DET', 1), ('ADJ', 2), ('NOUN', 2)],
                [('DET', 1), ('ADJ', 1), ('NOUN', 1), ('VERB', 1), ('ADJ', 1), ('NOUN', 2)],
                [('ADJ', 2), ('NOUN', 1), ('VERB', 1), ('ADJ', 1), ('NOUN', 2)]]
gather_words() # makes categories of words, listed by pos
bigram_collector(corpus) # makes list of bigrams
rhyme_dict = construct_rhyme_dict(corpus[:2000]) # make rhyme corpus
for i in range(20):
    print(str(i + 1) + ".\n")
    write_poem(template)